In [1]:
import numpy as np
import pandas as pd
import scipy.io
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
import os
import sys
import seaborn as sns

In [2]:
from google.colab import drive
drive.mount("/content/drive")
# @title Mount Drive

Mounted at /content/drive


In [3]:
cd /content/drive/MyDrive/Work/Sciencing/Google_CoLab_projects/WWD_Model/WWD_pytorch/github/linearfit_2p/

/content/drive/MyDrive/Work/Sciencing/Google_CoLab_projects/WWD_Model/WWD_pytorch/github/linearfit_2p


In [4]:
sys.path.append('code/')

In [5]:
from linearfit import LinearFit

In [6]:
base_dir =  '/content/drive/MyDrive/Work/Sciencing/Data/'

In [7]:
# data load


sc_file = base_dir +'sc_hcp1065_2mm.xlsx'
sc_df = pd.read_excel(sc_file)

In [8]:
# sc  template
sc = sc_df.values[:,1:].astype('float')
SC =(sc+sc.T)*0.5
       
sc = 0.5*np.log1p(SC)/np.linalg.norm(np.log1p(SC))



In [10]:
num_roi = 132
mask = np.tril_indices(2*num_roi, 0)
gee_lsd = []
for i in range(0, 50):#len(subs)):

    
    print(i)
    file_name = base_dir + 'Harvard-AAL-Segmen_Subject'+ '{0:03}'.format(i+1) +'_Condition001.mat'
    if os.path.isfile(file_name):
        test=scipy.io.loadmat(file_name)
        ts = np.ones((num_roi, 300))
        for i in range(num_roi):
            if test['conditionweights'][0,0][:300].mean() > 0:
                ts[i] = test['data'][0, i+3][:300, 0]
            else:
                ts[i] = test['data'][0, i+3][300:, 0]
        gee = 0.2
        gii = 0.5
        gie = 0.5
        gei = 0.5
        
        
        fc_emp = np.corrcoef(ts)
          
            
        #ts = np.loadtxt(ts_file)
        sc_a = np.zeros((num_roi*2, num_roi*2))
        fc_a = np.zeros((num_roi*2, num_roi*2))
        sc_a[:num_roi,:num_roi] = sc +0.015*(fc_emp) +gee*np.eye(num_roi)
        sc_a[num_roi:,num_roi:] = gii*np.eye(num_roi)#
        sc_a[:num_roi,num_roi:] = -0.01*np.eye(num_roi)
        sc_a[num_roi:,:num_roi] = 0.01*np.eye(num_roi)
        sc_a = sc_a/np.linalg.norm(sc_a)
        

        
        
        
        fc_a[:num_roi,:num_roi] = fc_emp
        fc_a[num_roi:,num_roi:] = .8*fc_emp #np.ones(num_roi)#
        fc_a[:num_roi,num_roi:] = 0.2*fc_emp#0.2*np.ones(num_roi)#
        fc_a[num_roi:,:num_roi] = 0.2*fc_emp#0.2*np.ones(num_roi)#
        std = 1.0e-4
        num_converge= 50
        
        model = LinearFit(sc_a, fc_a)
        model.ldm_2nd_model_fitting(std, num_converge)
        sc_fitted = model.eSC
        fc_sim = model.simFC

        fig, ax = plt.subplots(1,4)
        ax[0].imshow(fc_sim[:num_roi,:num_roi]- np.diag(np.diag(fc_sim[:num_roi,:num_roi])), cmap='bwr')
        
        ax[1].imshow(fc_emp, cmap='bwr')
        ax[2].imshow(sc, cmap='bwr')
        ax[3].imshow(sc_fitted- np.diag(np.diag(sc_fitted)), cmap='bwr')
        plt.show()

        print(np.corrcoef(fc_emp[np.tril_indices(num_roi,-1)], fc_sim[:num_roi,:num_roi][np.tril_indices(num_roi,-1)])[0,1])
        print('gie=', np.array([sc_fitted[i,num_roi+i] for i in range(num_roi)]).mean())
        print('gee=', np.array([sc_fitted[i,i] for i in range(num_roi)]).mean())
        #sc_fitted[np.abs(sc_fitted) < 0.5*np.abs(sc_fitted).mean()] = 0
        gee_lsd.append(sc_fitted.ravel())
        #gee_lsd.append(np.concatenate([np.diag(sc_fitted[:num_roi,num_roi:2*num_roi]), np.diag(sc_fitted[num_roi:2*num_roi,:num_roi])]))
        print('gii=', np.array([sc_fitted[num_roi+i,num_roi+i] for i in range(num_roi)]).mean())
        print('gei=', np.array([sc_fitted[num_roi+i,i] for i in range(num_roi)]).mean())
        print('simFC min:', fc_sim.min(), 'simFC max:', fc_sim.max())

Output hidden; open in https://colab.research.google.com to view.

In [11]:
num_roi = 132
gee_placibo = []
for i in range(0, 50):#len(subs)):

    
    print(i)
    file_name = base_dir + 'Harvard-AAL-Segmen_Subject'+ '{0:03}'.format(i+1) +'_Condition001.mat'
    if os.path.isfile(file_name):
        test=scipy.io.loadmat(file_name)
        ts = np.ones((num_roi, 300))
        for j in range(num_roi):
            if test['conditionweights'][0,0][:300].mean() > 0:
                ts[j] = test['data'][0, j+3][300:, 0]
            else:
                ts[j] = test['data'][0, j+3][:300, 0]
        gee = 0.2
        gii = 0.5
        gie = 0.5
        gei = 0.5
        
        
        fc_emp = np.corrcoef(ts)
          
            
        #ts = np.loadtxt(ts_file)
        sc_a = np.zeros((num_roi*2, num_roi*2))
        fc_a = np.zeros((num_roi*2, num_roi*2))
        sc_a[:num_roi,:num_roi] = sc +0.015*(fc_emp) +gee*np.eye(num_roi)
        sc_a[num_roi:,num_roi:] = gii*np.eye(num_roi)#
        sc_a[:num_roi,num_roi:] = -0.01*np.eye(num_roi)
        sc_a[num_roi:,:num_roi] = 0.01*np.eye(num_roi)
        sc_a = sc_a/np.linalg.norm(sc_a)
        

        
        
        
        fc_a[:num_roi,:num_roi] = fc_emp
        fc_a[num_roi:,num_roi:] = .8*fc_emp #np.ones(num_roi)#
        fc_a[:num_roi,num_roi:] = 0.2*fc_emp#0.2*np.ones(num_roi)#
        fc_a[num_roi:,:num_roi] = 0.2*fc_emp#0.2*np.ones(num_roi)#
        std = 1.0e-4
        num_converge= 50
        
        model = LinearFit(sc_a, fc_a)
        model.ldm_2nd_model_fitting(std, num_converge)
        sc_fitted = model.eSC
        fc_sim = model.simFC

        fig, ax = plt.subplots(1,4)
        ax[0].imshow(fc_sim[:num_roi,:num_roi]- np.diag(np.diag(fc_sim[:num_roi,:num_roi])), cmap='bwr')
        
        ax[1].imshow(fc_emp, cmap='bwr')
        ax[2].imshow(sc, cmap='bwr')
        ax[3].imshow(sc_fitted- np.diag(np.diag(sc_fitted)), cmap='bwr')
        plt.show()

        print(np.corrcoef(fc_emp[np.tril_indices(num_roi,-1)], fc_sim[:num_roi,:num_roi][np.tril_indices(num_roi,-1)])[0,1])
        print('gie=', np.array([sc_fitted[i,num_roi+i] for i in range(num_roi)]).mean())
        print('gee=', np.array([sc_fitted[i,i] for i in range(num_roi)]).mean())
        #sc_fitted[np.abs(sc_fitted) < 0.5*np.abs(sc_fitted).mean()] = 0
        gee_placibo.append(sc_fitted.ravel())
        #gee_placibo.append(np.concatenate([np.diag(sc_fitted[:num_roi,num_roi:2*num_roi]), np.diag(sc_fitted[num_roi:2*num_roi,:num_roi])]))
        print('gii=', np.array([sc_fitted[num_roi+i,num_roi+i] for i in range(num_roi)]).mean())
        print('gei=', np.array([sc_fitted[num_roi+i,i] for i in range(num_roi)]).mean())
        print('simFC min:', fc_sim.min())
        plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [12]:
data_fitting = np.concatenate([np.array(gee_lsd), np.array(gee_placibo)], axis=0)

In [13]:
data_fitting_file = base_dir + 'fittingresults_2p_lin_LSD_asym_15.txt'
np.savetxt(data_fitting_file, data_fitting)
